# CVAL on-premice YOLOv5 advanced example

## Install dependencies

In [ ]:
%cd .
!ls
# Optional for local:
# !apt-get install git -y
!git clone https://github.com/fangorntreabeard/yolov5.git
%cd yolov5
!pip install -q pydantic==1.10.9
!pip install -qr requirements.txt
!pip install -q cval-lib
!pip install -q git+https://github.com/fangorntreabeard/CLIP.git
%cd ..
!ls

In [ ]:
# set up your user_api_key

USER_API_KEY = ...

## Structure of data
    data
        dataset_name
            all_train_images
            all_train_labels
            train
                labels
                    annotations in YOLOV5 format
                images
            test
                labels
                    annotations in YOLOV5 format
                images
            val
                labels
                   annotations in YOLOV5 format
                images
        data/data.yml
    yolov5/yolov5s.pt




## Main class for experiments


In [ ]:
import os
import uuid
from pathlib import Path
from contextlib import suppress
import random
import json
import shutil
from typing import List
import time
from ultralytics import YOLO
from yolov5 import train, val, detect
from cval_lib.connection import CVALConnection


class YOLOWorker:
    # config
    def __init__(self, path_to_dataset: str, model: str = None):
        self.path_to_data = Path(path_to_dataset).parent
        self.dataset = Path(path_to_dataset).name
        self.dataset_conf = self.path_to_data / 'data.yaml'
        self.model = model
        self.weights = None
        self.all_train_images = self.path_to_data / self.dataset / 'all_train_images'

    # copy dataset/train/labels data
    def annotate_dataset(self, img_ids: List[str], random_sample = None):
        ids = set(
            map(
                lambda x: Path(x).name.replace('.txt', ''),
                os.listdir(
                    Path(self.path_to_data) / self.dataset / 'all_train_labels'
                )
            )
        ).intersection(set(img_ids))
        if random_sample is not None:
          ids = random.sample(list(ids), random_sample)
        labels = Path(self.path_to_data) / self.dataset / 'train/labels'
        images = Path(self.path_to_data) / self.dataset / 'train/images'
        # remove cache
        cache = Path(self.path_to_data) / self.dataset / 'train/labels.cache'
        counter = 0
        with suppress(Exception):
          shutil.rmtree(labels)
        with suppress(Exception):
          shutil.rmtree(images)
        with suppress(Exception):
          cache.unlink()
        os.makedirs(labels, exist_ok=True)
        os.makedirs(images, exist_ok=True)
        for i in ids:
            shutil.copyfile(
                Path(Path(self.path_to_data) / self.dataset / 'all_train_labels') / (i+'.txt'),
                Path(Path(self.path_to_data) / self.dataset / 'train/labels') / (i+'.txt'),
            )
            shutil.copyfile(
                Path(self.all_train_images) / (i+'.jpg'),
                Path(Path(self.path_to_data) / self.dataset / 'train/images') / (i+'.jpg'),
            )

    # start training
    def train(
            self,
            device: int = 0,
            batch: int = -1,
            log_tag: str = 'default',
            epochs: int = 100,
            pretrain: bool = False,ч
            workers=50,
            model: str = None,
    ):
        self.model = f'{uuid.uuid4()}' if model is None else model
        self.weights = self.path_to_data / self.dataset / f'calls/{self.model}/exp/weights/best.pt'
        train.run(
            workers=workers,
            weights='yolov5/yolov5s.pt',
            data=self.dataset_conf,
            epochs=epochs,
            pretrained=pretrain,
            nosave=False,
            batch=batch,
            device=device,
            project=self.path_to_data/ self.dataset / "calls" / self.model,
        )
        self.write_log(log_tag)

    # get test values
    def test(
        self,
        log_tag: str = 'default',
        device: int = 0,
        workers=50,
        batch_size=50,
    ):
        time.sleep(10)
        with suppress(FileNotFoundError):
            shutil.rmtree(self.path_to_data / self.dataset / f'calls/{self.model}/exp' / 'val')
        res, _, _ = val.run(
            weights=self.weights,
            data=self.dataset_conf,
            task='test',
            workers=workers,
            save_txt=True,
            save_conf=True,
            device=device,
            project=self.path_to_data/ self.dataset / "calls" / self.model,
            batch_size=batch_size,
        )
        self.write_log(log_tag)
        return res

    def write_log(self, log_tag):
        if not Path(f"{self.path_to_data}/logs").exists():
            os.makedirs(Path(f"{self.path_to_data}/logs"), exist_ok=True)
        with open(f"{self.path_to_data}/logs/{log_tag}.log", 'a+') as f:
            f.write(self.model + '\n')

    # get bbox scores
    def detect(
            self,
            device: int = 0,
            log_tag: str = 'default',
            rm=True,
    ):
        time.sleep(2)
        with suppress(Exception):
            os.remove('yolov5/confidence.txt')
        detect.run(
            data=self.dataset_conf,
            weights=self.weights,
            source=self.all_train_images,
            project=self.path_to_data/ self.dataset / "calls" / self.model,
            exist_ok=True,
            device=device,
            nosave=True,
            save_conf=True,
            save_txt=True,
        )
        self.write_log(log_tag)
        return self.bbox_score()

    # get IDs of images
    @property
    def ids(self):
        return list(map(lambda x: Path(x).name.replace('.jpg', ''), os.listdir(self.all_train_images)))

    @staticmethod
    def _score_parser(f):
        return list(map(lambda x: {'frame_id': x[0],  'category_id': int(x[1]), 'score': float(x[-1])}, [i.split('\t') for i in frozenset(f.readlines())]))

    def bbox_score(self):
        try:
            with open(f'yolov5/confidence.txt') as f:
                return self._score_parser(f)
        except:
            print("No detections!")
            return []

In [ ]:
import os
import pathlib as plb
import uuid
from typing import Union

import clip
import numpy as np
import torch
from PIL import Image

from cval_lib.models.embedding import FrameEmbeddingModel, EmbeddingModel

# embedding and score generator for schemes FrameEmbeddingModel, EmbeddingModel

def get_frames(imgs_path: Union[os.PathLike, str, plb.Path], weights_path: Union[os.PathLike, str, plb.Path]):
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model_yolo = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
    model_clip, preprocess = clip.load("RN101", device=device)
    names_imgs = os.listdir(imgs_path)
    list_imgs = [os.path.join(imgs_path, path) for path in names_imgs]
    tabs = []
    imgs_numpy = []

    for batch in range(len(list_imgs) // 10 + 1):
        bb = model_yolo(list_imgs[batch * 10: (batch + 1) * 10])
        tabs = tabs + [x.detach().cpu().numpy() for x in bb.xyxy]
        imgs_numpy = imgs_numpy + bb.ims
    n_images = len(tabs)
    for i in range(n_images):
        if len(tabs[i]) == 0:
            tabs[i] = np.array(
                [
                    [0, 0, imgs_numpy[i].shape[0], imgs_numpy[i].shape[1], 1, -1],
                ],
            )
    n_images = len(tabs)
    embeddings = []
    predictions = []

    for i in range(n_images):
        emb = []
        scr = []
        for boxs in range(len(tabs[i])):
            x1 = int(tabs[i][boxs, 0])
            y1 = int(tabs[i][boxs, 1])
            x2 = int(tabs[i][boxs, 2])
            y2 = int(tabs[i][boxs, 3])
            im = Image.fromarray(imgs_numpy[i][y1: y2, x1: x2])
            im.save('test.jpg')
            image = preprocess(Image.open("test.jpg")).unsqueeze(0).to(device)
            with torch.no_grad():
                _id = uuid.uuid4().hex
                image_features = model_clip.encode_image(image)
                image_features = [float(x) for x in image_features.detach().cpu().numpy()[0].tolist()]
                scr.append(
                    {
                        "embedding_id": _id,
                        "score": float(tabs[i][boxs, 4]) + 0.01,
                        "category_id": int(tabs[i][boxs, 5]),
                    },
                )
                emb.append(
                    EmbeddingModel(**{
                        "embedding_id": _id,
                        "embedding": image_features,
                    })
                )

        embeddings.append(
            FrameEmbeddingModel(**{
                "frame_id": names_imgs[i].split('.')[0],
                "embeddings": emb,
            }),
        )
        predictions.append(
            {
                "frame_id": names_imgs[i].split('.')[0],
                "predictions": scr,
            },
        )
    return embeddings, predictions

## Experiments

### Random sampling



In [ ]:
PATH_TO_DATASET = ...
LINES_COUNT = ...
DETECTOR = CVALConnection(USER_API_KEY).detection()
STEPS = ...

for _ in range(LINES_COUNT):
    yolo = YOLOWorker(PATH_TO_DATASET)
    train_size = len(yolo.ids)
    step = train_size // STEPS
    ids = []
    for sample_size in range(step, step*(STEPS+1), step):
        # create ranodm sample
        ids += random.sample(list(filter(lambda x: x not in ids, yolo.ids)), step)
        yolo.annotate_dataset(ids)
        yolo.train(
            epochs=100,
            workers=150,
        )
        with open('result_random.csv', 'a+') as f:
            f.write(f'{time.time()},{",".join(list(map(str, yolo.test(workers=1))},{sample_size};\n')

### Entropy sampling

In [ ]:
from cval_lib.models.detection import BBoxScores, FramePrediction
from cval_lib.models.detection import DetectionSamplingOnPremise

PATH_TO_DATASET = ...
LINES_COUNT = ...
DETECTOR = CVALConnection(USER_API_KEY).detection()
STEPS = ...

for _ in range(LINES_COUNT):
    yolo = YOLOWorker(PATH_TO_DATASET)
    ids = yolo.ids
    train_size = len(ids)
    step = train_size // STEPS
    ids = random.sample(ids, step)
    yolo.annotate_dataset(ids)

    for sample_size in range(step, step*(STEPS+1), step):
        delta = []
        yolo.train(epochs=100, workers=1, pretrain=True)
        od_res = [i for i in yolo.detect() if i not in ids]
        # if YOLOV5 detections are not empty, create request with predictions
        if od_res:
            frames_pred = list(map(lambda x: {'frame_id': x, 'predictions': []}, set(filter(lambda x: x not in ids, map(lambda x: x.get('frame_id'), od_res)))))
            for (i, frame) in enumerate(frames_pred):
                frames_pred[i]['predictions'] = frames_pred[i]['predictions'] + list(
                    map(lambda x: x, filter(lambda x: x.get('frame_id') == frame.get('frame_id'), od_res))
                )
            frames_pred = list(filter(lambda x: x.get('predictions'), frames_pred))
            request = DetectionSamplingOnPremise(
                 num_of_samples=step if step <= len(frames_pred) else len(frames_pred),
                 bbox_selection_policy='sum',
                 selection_strategy='entropy',
                 sort_strategy='ascending',
                 frames=frames_pred,
                )
            delta = list(map(lambda x: x, DETECTOR.on_premise_sampling(request).result))
        # add random
        delta += random.sample(list(set(yolo.ids) - set(delta) - set(ids)), step-len(delta))
        # add delta
        # write result
        with open('result_entropy.csv', 'a+') as f:
            f.write(f'{time.time()},{",".join(list(map(str, yolo.test(workers=1))))},{sample_size};\n')
        ids += delta
        yolo.annotate_dataset(ids)

### Clustering sampling

In [ ]:
from cval_lib.connection import CVALConnection

from cval_lib.models.detection import BBoxScores, FramePrediction
from cval_lib.models.detection import DetectionSamplingOnPremise
from cval_lib.connection import CVALConnection

PATH_TO_DATASET = ...
YOLO = YOLOWorker(PATH_TO_DATASET)
PATH_TO_ALL_TRAIN_IMAGES = YOLO.all_train_images
LINES_COUNT = ...
DETECTOR = CVALConnection(USER_API_KEY)
STEPS = ...

for _ in range(LINES_COUNT):
    ids = YOLO.ids
    step = 250
    ids = random.sample(ids, step)
    YOLO.annotate_dataset(ids)
    YOLO.train(epochs=epoch, pretrain=True)
    ds_id = detector.dataset().create(name='b', description='b')
    for sample_size in range(step, step * STEPS, step):
        embeddings, predictions = get_frames(YOLO.all_train_images, YOLO, ids)
        if len(predictions) > step:
            detector.embedding(dataset_id=ds_id, part_of_dataset='training').upload_many(embeddings)
            result = detector.detection().on_premise_sampling(
                DetectionSamplingOnPremise(
                 num_of_samples=step,
                 selection_strategy='clustering',
                 dataset_id=ds_id,
                 frames=predictions,
                )
            )

            while result.result is None:
                result = detector.result().get(result.task_id)
            ids += list(
                filter(lambda i: i not in ids, result.result),
            )
        else:
            good_id = [x["frame_id"] for x in predictions]
            delta = good_id + random.sample(list(set(YOLO.ids) - set(good_id) - set(ids)), k=step-len(good_id))

            ids += delta
        with open('result_cluster.csv', 'a+') as f:
            f.write(f'{time.time()},{",".join(list(map(str, YOLO.test())))},{sample_size};\n')
        YOLO.annotate_dataset(ids)
        if sample_size != step * (STEPS-1):
            YOLO.train(epochs=epoch, pretrain=True)

### Results

In [ ]:
# get results

from pathlib import Path

for pth in (
    'result_clustering.csv',
    'result_entropy.csv',
    'result_random.csv',
):
  if Path(pth).exists():
    with open(pth) as res:
      print(
            pth,
            res.read(),
            sep=f"\n{'_'*len(pth)}\n\n",
      )